<a href="https://colab.research.google.com/github/NikkiBoss-8/COMP30770-Project/blob/main/Project_of_COMP30770_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Install Required Libraries
Start by installing any necessary libraries (e.g., PySpark). This ensures that your environment is set up correctly.

In [1]:
# Install PySpark
!pip install pyspark

2. Import Libraries
Import all the required libraries at the beginning of your notebook.

In [2]:
from pyspark import SparkContext
import numpy as np
import os
import time

3. Traditional Solution
Include the code for your traditional solution (without MapReduce).

In [3]:
#Decomposition of Tasks: Data Cleaning: Check for missing or invalid values.
!awk -F',' 'NF==14' ItalianWine.csv > cleaned_data.cs

In [4]:
#Feature Scaling:   Scale numerical features to have zero mean and unit variance.
%%writefile mapper.py

 # !/usr/bin/python3

from sklearn.preprocessing import StandardScaler
import pandas as pd

data = pd.read_csv("ItalianWine.csv")
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data.iloc[:, :-1])


Overwriting mapper.py


In [5]:
#Dimensionality Reduction:  Apply Principal Component Analysis (PCA) to reduce the number of features.
%%writefile mapper.py

# !/usr/bin/python3
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
reduced_data = pca.fit_transform(scaled_data)

Overwriting mapper.py


In [6]:
#Model Training:  Train a Random Forest classifier to predict wine classes.
%%writefile mapper.py

# !/usr/bin/python3
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reduced_data, data['label'], test_size=0.2)
model = RandomForestClassifier()
model.fit(X_train, y_train).show()

Overwriting mapper.py


In [16]:
# Loading our hourly data into a PySpark DataFrame:
df = spark.read.options(delimiter=",", header=True, inferSchema=True).csv("/content/ItalianWine.csv")

In [17]:
# We can see some more information about our data:
# The inferSchema automatically guesses the datatypes
# Otherwise it would assume all datatypes are strings
df.printSchema()

root
 |-- Alcohol: double (nullable = true)
 |-- Malic acid: double (nullable = true)
 |-- Ash: double (nullable = true)
 |-- Alcalinity of ash: double (nullable = true)
 |-- Magnesium: double (nullable = true)
 |-- Total phenols: double (nullable = true)
 |-- Flavanoids: double (nullable = true)
 |-- Nonflavanoid phenols: double (nullable = true)
 |-- Proanthocyanins: double (nullable = true)
 |-- Color intensitys: double (nullable = true)
 |-- Hue: double (nullable = true)
 |-- OD280/OD315 of diluted wines: double (nullable = true)
 |-- Proline: double (nullable = true)
 |-- label: integer (nullable = true)



In [19]:
#Model Evaluation:    Evaluate the model using accuracy and confusion matrix.
%%writefile mapper.py

# !/usr/bin/python3
from sklearn.metrics import accuracy_score, confusion_matrix

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

Overwriting mapper.py


4. MapReduce Solution
Include the code for your MapReduce solution.

In [20]:
#As Spark is written in the Scala programming language, it requires the JVM to run. So our first step is to install Java:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [21]:
#Next we need to download and upack Apache Spark and Hadoop:
!wget https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!rm spark-3.3.1-bin-hadoop3.tgz   # Tidying up


--2025-03-16 17:06:57--  https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299350810 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop3.tgz’

spark-3.3.1-bin-had 100%[===================>] 285.48M  18.5MB/s    in 16s     

2025-03-16 17:07:14 (17.3 MB/s) - ‘spark-3.3.1-bin-hadoop3.tgz’ saved [299350810/299350810]



In [22]:
# Setting up our environmental variables:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [23]:
#Next we'll need to install the findpark library to locate Spark on the system:
!pip install -q findspark
import findspark
findspark.init()

In [24]:
#We can now import SparkSession from pyspark.sql to create our entry point to Spark.
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) #  This will format our output tables a bit nicer when not using the show() method
spark

In [25]:
#You can check how many CPU cores are available to you.
import multiprocessing
print(multiprocessing.cpu_count())

2


In [29]:
#Upload the File to Colab: if nessecarey

from google.colab import files
uploaded = files.upload()

Saving ItalianWine.csv to ItalianWine (1).csv


In [30]:
#Verify the File Path:
!ls /content

 cleaned_data.cs        ItalianWine.csv   output_scaled_data_1742141345   spark-3.3.1-bin-hadoop3
'ItalianWine (1).csv'   mapper.py	  sample_data


In [35]:
#Use the Correct File Path:
!data = sc.textFile("/content/ItalianWine.csv")

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `data = sc.textFile("/content/ItalianWine.csv")'


In [40]:
# Loading our hourly data into a PySpark DataFrame:
df = spark.read.options(delimiter=",", header=True, inferSchema=True).csv("/content/ItalianWine.csv")
# First we'll show just the first 10 records:
df.select("*").show(10)

+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+----------------+----+----------------------------+-------+-----+
|Alcohol|Malic acid| Ash|Alcalinity of ash|Magnesium|Total phenols|Flavanoids|Nonflavanoid phenols|Proanthocyanins|Color intensitys| Hue|OD280/OD315 of diluted wines|Proline|label|
+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+----------------+----+----------------------------+-------+-----+
|  14.23|      1.71|2.43|             15.6|    127.0|          2.8|      3.06|                0.28|           2.29|            5.64|1.04|                        3.92| 1065.0|    1|
|   13.2|      1.78|2.14|             11.2|    100.0|         2.65|      2.76|                0.26|           1.28|            4.38|1.05|                         3.4| 1050.0|    1|
|  13.16|      2.36|2.67|             18.6|    101.0|          2.8|      3.24|                 

In [72]:
#Check if the file exists and is accessible
import os

file_path = "/content/ItalianWine.csv"
if os.path.exists(file_path):
    print(" File exists:", file_path)
else:
    print(" File NOT found. Check the file path!")


 File exists: /content/ItalianWine.csv


In [57]:
#Verify Java Installation Check if Java is installed correctly:
!java -version

openjdk version "11.0.26" 2025-01-21
OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)


In [74]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

In [75]:
!ls /content
#Fix data = sc.textFile("/content/ItalianWine.csv")
#Make sure the dataset actually exists and is properly formatted. Try checking the file:

 cleaned_data.cs	   ItalianWine.csv		   sample_data
'ItalianWine (1).csv'	   mapper.py			   spark-3.3.1-bin-hadoop3
 ItalianWine_cleaned.csv   output_scaled_data_1742141345


In [76]:
!head -n 5 /content/ItalianWine.csv
#If ItalianWine.csv is missing or empty, re-upload it and verify the contents using:

Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensitys,Hue,OD280/OD315 of diluted wines,Proline,label
14.23,1.71,2.43,15.6,127.0,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,1
13.2,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.4,1050.0,1
13.16,2.36,2.67,18.6,101.0,2.8,3.24,0.3,2.81,5.68,1.03,3.17,1185.0,1
14.37,1.95,2.5,16.8,113.0,3.85,3.49,0.24,2.18,7.8,0.86,3.45,1480.0,1


In [77]:
# If the file is malformed, load it using Pandas and write it back:
import pandas as pd

df = pd.read_csv("/content/ItalianWine.csv")
df.to_csv("/content/ItalianWine_cleaned.csv", index=False)


In [78]:
data = sc.textFile("/content/ItalianWine_cleaned.csv")
#Then, update your Spark code:

In [79]:
#Ensure PySpark is Running Properly Since PySpark runs on Java, confirm the environment is working by running:


from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Test").getOrCreate()
print("Spark is running!")
spark.stop()


Spark is running!


In [66]:
# Import necessary libraries
from pyspark.sql import SparkSession

# Ensure previous Spark sessions are stopped before creating a new one
spark = SparkSession.builder.appName("ItalianWineAnalysis").getOrCreate()

# Load the CSV file correctly using Spark DataFrame
df_spark = spark.read.csv("/content/ItalianWine.csv", header=True, inferSchema=True)

# Show the first few rows
df_spark.show(10)


+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+----------------+----+----------------------------+-------+-----+
|Alcohol|Malic acid| Ash|Alcalinity of ash|Magnesium|Total phenols|Flavanoids|Nonflavanoid phenols|Proanthocyanins|Color intensitys| Hue|OD280/OD315 of diluted wines|Proline|label|
+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+----------------+----+----------------------------+-------+-----+
|  14.23|      1.71|2.43|             15.6|    127.0|          2.8|      3.06|                0.28|           2.29|            5.64|1.04|                        3.92| 1065.0|    1|
|   13.2|      1.78|2.14|             11.2|    100.0|         2.65|      2.76|                0.26|           1.28|            4.38|1.05|                         3.4| 1050.0|    1|
|  13.16|      2.36|2.67|             18.6|    101.0|          2.8|      3.24|                 

In [89]:
# Install PySpark if not already installed
!pip install pyspark

# Import necessary libraries
from pyspark.sql import SparkSession
import numpy as np
import time
import os
import time

start_time = time.time()


# Create a Spark session (prevents multiple SparkContexts issue)
spark = SparkSession.builder.appName("ItalianWineAnalysis").getOrCreate()
sc = spark.sparkContext  # Get SparkContext from SparkSession

# File path for the dataset
file_path = "/content/ItalianWine.csv"

# Ensure the dataset exists
if not os.path.exists(file_path):
    raise FileNotFoundError(f"File not found: {file_path}")

# Load the dataset
data = sc.textFile(file_path)

# Extract and remove the header safely
#header = data.first()
data = data.filter(lambda row: row != header)

# Function to scale features
def scale_features(row):
    try:
        # Convert row into features (excluding the label)
        values = row.split(',')
        features = np.array(values[:-1], dtype=float)
        label = values[-1]  # Extract label (assumed last column)

        # Compute mean and standard deviation safely
        mean = np.mean(features)
        std = np.std(features) if np.std(features) != 0 else 1  # Prevent division by zero

        # Apply feature scaling (zero mean, unit variance)
        scaled_features = (features - mean) / std

        # Return formatted row with scaled features
        return ','.join(map(str, scaled_features)) + ',' + label
    except Exception as e:
        return f"Error processing row: {row} -> {str(e)}"

# Apply feature scaling transformation
scaled_data = data.map(scale_features)

# Filter out any errors that may have occurred during processing
scaled_data = scaled_data.filter(lambda row: not row.startswith("Error"))

# Generate a unique output directory name using a timestamp
output_dir = f"/content/output_scaled_data_{int(time.time())}"

# Save the transformed dataset
scaled_data.saveAsTextFile(output_dir)
print(f"Processed data saved to: {output_dir}")

# Stop SparkSession (no need to stop SparkContext explicitly)
spark.stop()
end_time = time.time()
print(f"Execution Time: {end_time - start_time} seconds")

UsageError: Line magic function `%%writefile` not found.


5. View Results
Add a section to view the results of your processing. This helps verify that the code is working as expected.

In [90]:
# Import necessary libraries
from pyspark.sql import SparkSession

# Ensure previous Spark sessions are stopped before creating a new one
spark = SparkSession.builder.appName("ItalianWineAnalysis").getOrCreate()

# Load the CSV file correctly using Spark DataFrame
df_spark = spark.read.csv("/content/ItalianWine.csv", header=True, inferSchema=True)

# Show the first few rows
df_spark.show(10)

+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+----------------+----+----------------------------+-------+-----+
|Alcohol|Malic acid| Ash|Alcalinity of ash|Magnesium|Total phenols|Flavanoids|Nonflavanoid phenols|Proanthocyanins|Color intensitys| Hue|OD280/OD315 of diluted wines|Proline|label|
+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+----------------+----+----------------------------+-------+-----+
|  14.23|      1.71|2.43|             15.6|    127.0|          2.8|      3.06|                0.28|           2.29|            5.64|1.04|                        3.92| 1065.0|    1|
|   13.2|      1.78|2.14|             11.2|    100.0|         2.65|      2.76|                0.26|           1.28|            4.38|1.05|                         3.4| 1050.0|    1|
|  13.16|      2.36|2.67|             18.6|    101.0|          2.8|      3.24|                 

6. Performance Comparison
Add a section to compare the performance of the traditional solution and the MapReduce solution. You can use Python's time module to measure execution time.

In [91]:
import time

# Traditional Solution
start_time = time.time()
# Insert traditional solution code here
end_time = time.time()
print(f"Traditional Solution Execution Time: {end_time - start_time} seconds")

# MapReduce Solution
start_time = time.time()
# Insert MapReduce solution code here
end_time = time.time()
print(f"MapReduce Solution Execution Time: {end_time - start_time} seconds")

Traditional Solution Execution Time: 2.8371810913085938e-05 seconds
MapReduce Solution Execution Time: 2.574920654296875e-05 seconds
